In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

In [3]:
import os
import random
import sys

src_dir = './../src/'
sys.path[0] = src_dir

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from copy import deepcopy
from scipy.stats import spearmanr


from access_biology_data import meta
from aging_tools import inout

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
sys.path.append('./../src/')
from aging_tools import inout, export
from access_aging_data import chaperome, earlier_studies, companions, sequencing
from access_biology_data import annotation, relations

# Construct sanity check: get median expression for every gene

In [6]:
# Get aging expression data, filter, and map to ncbi
df_counts, df_meta, df_genes = sequencing.load_cached_aging_map(
    dataset_name='aging_map_tmm_180105',
    unambiguous_to_entrez=True,
    as_entrez=True
)

In [7]:
%%time
agg = []
for pfu in df_meta['pfu'].unique():
    f_pfu = df_meta['pfu']==pfu
    for tissue in df_meta['tissue'].unique():
        f_tissue = df_meta['tissue']==tissue
        for age in df_meta['age'].unique():
            f_age = df_meta['age']==age
            f = f_pfu & f_tissue & f_age
            
            if any(f):
                d = df_counts.loc[:, f]
                d = d.median(1).to_frame('median')
                d.loc[:, 'tissue'] = tissue
                d.loc[:, 'pfu'] = pfu
                d.loc[:, 'age'] = age
                d = d.reset_index()

                agg.append(d)
                
df_median_counts = pd.concat(agg)                

CPU times: user 1.07 s, sys: 140 ms, total: 1.21 s
Wall time: 1.22 s


In [8]:
import glob

In [9]:
agg = []

for batch in [1, 2, 3, 4, 5, 6]:
    p = inout.get_internal_path(
        'dynamic/tstoeger/181022_inclusive_any_detected_per_batch/{}/DE/Flu/*.csv'.format(int(batch)))
    d = glob.glob(p)

    files_to_process = pd.DataFrame(columns=['path'], data = d)
    files_to_process['base_name'] = files_to_process['path'].str.extract('.*/(.*).csv', expand=False)
    files_to_process[['tissue', 'pfu', 'dividend', 'divisor']] = files_to_process['base_name'].str.extract(
        '^(.*)_pfu_(.*)_ages_(.*) (.*)_DE', expand=False)
    files_to_process = files_to_process.set_index('base_name', verify_integrity=True)

    for j, v in files_to_process.iterrows():

        df = pd.read_csv(v['path'], usecols=['Symbol', 'log2FoldChange', 'pvalue', 'padj'])
        tags = ['tissue', 'pfu', 'dividend', 'divisor']
        for tag in tags:
            df.loc[:, tag] = v[tag]

        df.loc[:, 'batch'] = batch
            
        agg.append(df)

In [10]:
df = pd.concat(agg, axis=0)
df = df.rename(columns={'Symbol': 'gene_ensembl'})

for x in ['pfu', 'dividend', 'divisor']:
    df.loc[:, x] = df.loc[:, x].apply(float)

In [11]:
# Add ncbi gene IDs
df = pd.merge(
    df,
    df_genes[['gene_ensembl', 'gene_ncbi']], how='left').set_index('gene_ncbi').reset_index()

# Add some sanity checks and manually check discrepancies

In [12]:
df.loc[:, 'oldest'] = df.loc[:, ['dividend', 'divisor']].max(1)
df.loc[:, 'youngest'] = df.loc[:, ['dividend', 'divisor']].min(1)

In [13]:
for e in ['dividend', 'divisor', 'oldest', 'youngest']:
    df = pd.merge(
        df, 
        df_median_counts.rename(columns={'median': 'median_{}'.format(e)}), 
        left_on=['gene_ncbi', 'tissue', 'pfu', e],
        right_on=['gene_ncbi', 'tissue', 'pfu', 'age'],
        how='left'
    ).drop('age', 1)

In [14]:
f = df['dividend'] > df['divisor']

In [15]:
df.loc[f, 'o_over_y'] = df.loc[f, 'log2FoldChange']
df.loc[~f, 'o_over_y'] = -df.loc[~f, 'log2FoldChange']

In [16]:
test_dummy = df[(df['padj']<0.05) & ~(df['median_oldest'] == df['median_youngest'])]

In [17]:
f = test_dummy['median_oldest'] > test_dummy['median_youngest']
h = test_dummy[f]

In [18]:
h[h['o_over_y']<0].groupby(['tissue', 'pfu']).size()

tissue     pfu  
AM         0.0       33
           10.0      12
           150.0     18
AT2        0.0        4
           10.0      63
           150.0      7
Adrenal    0.0       12
BAT        0.0        7
Blood      0.0        3
           10.0       5
           150.0     17
Brain      0.0      130
Esophagus  0.0        8
Heart      0.0        1
Kidney     0.0       23
LI         0.0        3
Liver      0.0        1
Lung       0.0       66
           10.0      10
MoDC       0.0        1
           10.0       1
           150.0      5
MuscSat    0.0        8
SI         0.0        2
Skin       0.0      124
Stomach    0.0        2
WAT        0.0        2
dtype: int64

In [19]:
h[h['o_over_y']>0].groupby(['tissue', 'pfu']).size()

tissue      pfu  
AM          0.0       535
            10.0      125
            150.0     417
AT2         0.0       551
            10.0     1006
            150.0     344
Adrenal     0.0      1650
BAT         0.0      1307
Blood       0.0       340
            10.0     1033
            150.0     704
Brain       0.0       621
Cerebellum  0.0       102
Esophagus   0.0       744
GutEP       0.0       247
Heart       0.0       118
Kidney      0.0      5873
LI          0.0      1809
Liver       0.0        81
Lung        0.0       706
            10.0      501
            150.0     212
MoDC        0.0       280
            10.0       55
            150.0     359
MuscSat     0.0       157
SI          0.0       309
Skin        0.0      1310
Stomach     0.0      1474
WAT         0.0       763
dtype: int64

In [20]:
h[h['o_over_y']<0]

,gene_ncbi,gene_ensembl,log2FoldChange,pvalue,padj,tissue,pfu,dividend,divisor,batch,oldest,youngest,median_dividend,median_divisor,median_oldest,median_youngest,o_over_y
18370,213391.0,ENSMUSG00000042129,-0.466398,7.545895e-05,0.013110,AM,0.0,24.0,9.0,1,24.0,9.0,743.584262,742.983107,743.584262,742.983107,-0.466398
18378,72446.0,ENSMUSG00000032841,-0.898626,2.624856e-04,0.031609,AM,0.0,24.0,9.0,1,24.0,9.0,84.951081,79.347750,84.951081,79.347750,-0.898626
18385,71684.0,ENSMUSG00000036249,-1.210527,3.998168e-04,0.042574,AM,0.0,24.0,9.0,1,24.0,9.0,35.994294,31.299472,35.994294,31.299472,-1.210527
188939,11861.0,ENSMUSG00000047446,-0.606757,9.527012e-04,0.032941,BAT,0.0,9.0,4.0,1,9.0,4.0,164.241731,161.762762,164.241731,161.762762,-0.606757
264016,69833.0,ENSMUSG00000033020,-8.195262,2.470531e-05,0.014035,MuscSat,0.0,12.0,4.0,1,12.0,4.0,30.700600,19.848057,30.700600,19.848057,-8.195262
264019,71675.0,ENSMUSG00000042208,-8.066146,4.464556e-05,0.022348,MuscSat,0.0,12.0,4.0,1,12.0,4.0,13.303593,12.345507,13.303593,12.345507,-8.066146
264022,12909.0,ENSMUSG00000025532,-8.195041,8.834433e-05,0.036343,MuscSat,0.0,12.0,4.0,1,12.0,4.0,15.350300,7.005197,15.350300,7.005197,-8.195041
264023,12825.0,ENSMUSG00000026043,-1.207501,8.539539e-05,0.036343,MuscSat,0.0,12.0,4.0,1,12.0,4.0,1420.414439,1054.995253,1420.414439,1054.995253,-1.207501
264032,70984.0,ENSMUSG00000031938,-7.791774,1.618390e-04,0.049509,MuscSat,0.0,12.0,4.0,1,12.0,4.0,21.490420,14.814608,21.490420,14.814608,-7.791774
279888,227731.0,ENSMUSG00000026819,-1.624482,2.238276e-06,0.003593,Liver,0.0,24.0,12.0,1,24.0,12.0,69.523961,66.056197,69.523961,66.056197,-1.624482


In [21]:
f = test_dummy['median_oldest'] < test_dummy['median_youngest']
h = test_dummy[f]

In [22]:
h[h['o_over_y']<0].groupby(['tissue', 'pfu']).size()

tissue      pfu  
AM          0.0       304
            10.0       70
            150.0     295
AT2         0.0       513
            10.0      504
            150.0     288
Adrenal     0.0      1319
BAT         0.0       743
Blood       0.0       274
            10.0      279
            150.0     308
Brain       0.0       516
Cerebellum  0.0       234
Esophagus   0.0       615
GutEP       0.0       314
Heart       0.0       159
Kidney      0.0      4882
LI          0.0      1499
Liver       0.0        97
Lung        0.0       909
            10.0      234
            150.0     250
MoDC        0.0       324
            10.0       54
            150.0     460
MuscSat     0.0        92
SI          0.0       182
Skin        0.0      1527
Stomach     0.0       792
WAT         0.0       755
dtype: int64

In [23]:
h[h['o_over_y']>0].groupby(['tissue', 'pfu']).size()

tissue     pfu  
AM         0.0       24
           10.0      18
           150.0     23
AT2        10.0     237
           150.0      6
Adrenal    0.0        2
BAT        0.0       22
Blood      0.0        1
           10.0      22
           150.0     14
Brain      0.0      287
Esophagus  0.0        3
Heart      0.0        4
Kidney     0.0       36
LI         0.0        2
Liver      0.0        2
Lung       0.0       20
           10.0      25
           150.0      5
MoDC       0.0        7
           10.0       2
           150.0      2
MuscSat    0.0       17
Skin       0.0      175
Stomach    0.0        3
dtype: int64

# Finally: export

In [24]:
p = inout.get_internal_path('datasets/tstoeger/181024_pool_differential_expression_inclusive_any_detected_by_batch/age_groups.csv.gz')
inout.ensure_presence_of_directory(p)
df.to_csv(p, compression='gzip', index=False)